In [ ]:
import time
import random
import numpy as np
import csv
#matrices are none other than lists of lists
matrix_A = [
    [1,2,3],
    [4,5,6],
    [7,8,9]
]
#keeping in mind that python starts to count at 0
matrix_B = [
    [4,5,3],
    [3,4,8],
    [1,2,1]
]
for rows_a in matrix_A:#iterating through rows
    #print(rows_a)
    for element_a in rows_a:#iterating through elements
        #print(element_a)
        pass
 #matrix multiplication is row * column so I need a way to iterate through columns
# this function basically iterates through the rows 
# and takes the "Index" element of a row and puts its in a new list, which gives us a column
def matrix_column(Matrix, Index):
    column = []
    for row in Matrix:
        column.append(row[Index])
    return column
#as we can can get rows easily with a for loop, there is no reason to make a function

#now that we can iterate through rows and columns we can multiply   
def matrix_multiplication(A, B):
    result = [] #result matrix
    for row in A: #iterate through rows
        row_result = [] 
        for column_index in range(len(B[0])): #iterate through columns
            column = matrix_column(B, column_index)
            product_sum = 0
            for i in range(len(row)): #iterates through indices of the elements
                product_sum += row[i] * column[i]
            row_result.append(product_sum) # appends the sum of the products 
        result.append(row_result) # appends result rows to the matrix
    return result

def gen_matrices(size):
    matrix = []
    for k in range(size):#for columns
        row = []
        for i in range(size):
            row.append(random.randint(-99,99)) #generates rows that are "size" long
        matrix.append(row)#appends there rows to the matrix
    return matrix



with open('size_speed_comparison.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # header
    writer.writerow(['size', 'Own Function', 'NumPy'])
    
    
    for i in range(1, 11):
        size = 2**i
        A = gen_matrices(size)
        B = gen_matrices(size)

        start_time = time.time()
        multiplication = matrix_multiplication(A, B)
        end_time = time.time()
        computing_time = (end_time - start_time)

        np_A = np.array(A)
        np_B = np.array(B)
        start_time = time.time()
        np_multiplication = np.matmul(np_A, np_B)
        end_time = time.time()
        np_computing_time = (end_time - start_time)

        print(f"size: {size} own function: {computing_time}s , numpy: {np_computing_time}s")
        writer.writerow([size, computing_time, np_computing_time])
with open('amount_speed_comparison.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    #header
    writer.writerow(['Multiplications', 'Own Function Time', 'NumPy Time'])
    
    amount_of_multiplications = [1, 2, 5, 10, 50, 100, 300, 500, 1000]
    size = 2

    for num_multiplications in amount_of_multiplications:
        A = gen_matrices(size)
        B = gen_matrices(size)
        start_time = time.time()
        for _ in range(num_multiplications):
            multiplication = matrix_multiplication(A, B)
        end_time = time.time()
        custom_computing_time = end_time - start_time

        np_A = np.array(A)
        np_B = np.array(B)
        start_time = time.time()
        for _ in range(num_multiplications):
            np_multiplication = np.matmul(np_A, np_B)
        end_time = time.time()
        np_computing_time = end_time - start_time

        print(f"Multiplications: {num_multiplications}, Custom Time: {custom_computing_time}s, NumPy Time: {np_computing_time}s")
        writer.writerow([num_multiplications, custom_computing_time, np_computing_time])

print("done")

In [ ]:
import numpy as np
import random
import time
import csv

#I will use NumPy to write code more efficient. 

def gen_matrices(size):
    return np.random.randint(-99, 99, size=(size, size))

def simple_matrix_multiplication(A, B):
    return np.dot(A, B)

#split matrix into 4
def split(matrix):
    n = matrix.shape[0]
    a = matrix[:n//2, :n//2]
    b = matrix[:n//2, n//2:]
    c = matrix[n//2:, :n//2]
    d = matrix[n//2:, n//2:]
    return a, b, c, d

def strassens_algo(A, B):
#if its 2 or smaller we can use normal matrix multiplication
    if A.shape[0] <= 2:
        return simple_matrix_multiplication(A, B)
    
    # split both matrices into 4
    a, b, c, d = split(A)
    e, f, g, h = split(B)
    
    # recursive computing
    p1 = strassens_algo(a + d, e + h)
    p2 = strassens_algo(d, g - e)
    p3 = strassens_algo(a + b, h)
    p4 = strassens_algo(b - d, g + h)
    p5 = strassens_algo(a, f - h)
    p6 = strassens_algo(c + d, e)
    p7 = strassens_algo(a - c, e + f)
    
    # combine subsubmatrices back together
    C11 = p1 + p2 - p3 + p4
    C12 = p5 + p3
    C21 = p6 + p2
    C22 = p5 + p1 - p6 - p7
    
    # combine submatrices to get the resulting matrix
    top = np.hstack((C11, C12))
    bottom = np.hstack((C21, C22))
    return np.vstack((top, bottom))

# not recursive
def strassens_no_rec(A, B):
    a, b, c, d = split(A)
    e, f, g, h = split(B)
    
    p1 = simple_matrix_multiplication(a + d, e + h)
    p2 = simple_matrix_multiplication(d, g - e)
    p3 = simple_matrix_multiplication(a + b, h)
    p4 = simple_matrix_multiplication(b - d, g + h)
    p5 = simple_matrix_multiplication(a, f - h)
    p6 = simple_matrix_multiplication(c + d, e)
    p7 = simple_matrix_multiplication(a - c, e + f)
    
    C11 = p1 + p2 - p3 + p4
    C12 = p5 + p3
    C21 = p6 + p2
    C22 = p5 + p1 - p6 - p7
    
    top = np.hstack((C11, C12))
    bottom = np.hstack((C21, C22))
    return np.vstack((top, bottom))

# Speed-Test
with open('size_speed_comparison.csv', mode='r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Step 2: Add new columns
header = rows[0]  # Extract the header
header.extend(["Strassen's Optimized", "Another Algorithm"])  # Add new column names

for row in rows[1:]:  # Skip the header row
    size = int(row[0])
    A = gen_matrices(size)
    B = gen_matrices(size)

    # Compute new values
    start_time = time.time()
    optimized_result = strassens_algo(A, B)
    end_time = time.time()
    optimized_time = end_time - start_time

    start_time = time.time()
    another_result = strassens_no_rec(A, B)
    end_time = time.time()
    another_time = end_time - start_time
    print(f"size: {size} strassens: {optimized_time}s , no rec: {another_time}s")
    # Append results to the row
    row.extend([optimized_time, another_time])

with open('size_speed_comparison.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)
    


with open('amount_speed_comparison.csv', mode='r') as file:
    reader = csv.reader(file)
    rows = list(reader)

header = rows[0]  # extract the existing header
header.extend(["Strassen's Optimized Time", "Strassen's No Rec Time"])

for row in rows[1:]:  # skip the header row
    num_multiplications = int(row[0])  
    size = 2  

    A = gen_matrices(size)
    B = gen_matrices(size)

    start_time = time.time()
    for _ in range(num_multiplications):
        optimized_result = strassens_algo(A, B)
    end_time = time.time()
    optimized_time = end_time - start_time

    start_time = time.time()
    for _ in range(num_multiplications):
        another_result = strassens_no_rec(A, B)
    end_time = time.time()
    another_time = end_time - start_time

    print(f"Multiplications: {num_multiplications}, Strassen's Optimized Time: {optimized_time:.6f}s, Strassen's No Rec Time: {another_time:.6f}s")
    
    #append results to the current row
    row.extend([optimized_time, another_time])

#write the updated data back to the CSV
with open('amount_speed_comparison.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)